# Preprocessing

## Master Data

In [30]:
import pandas as pd

df_players_raw = pd.read_csv("../../data/raw/players/player_master.csv")
df_players_raw.head()


,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,NaN,NaN,NaN,NaN,NaN,Y,0
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,NaN,NaN,NaN,NaN,NaN,Y,0
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,NaN,NaN,NaN,NaN,NaN,Y,0
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,NaN,NaN,NaN,NaN,NaN,Y,0
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,NaN,NaN,NaN,NaN,NaN,Y,0


In [38]:
# Clean column names for consistency
df_players = df_players_raw.copy()
df_players.columns = (
    df_players.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("/", "_")
)

# Rename key columns to clearer names
df_players = df_players.rename(columns={
    "person_id": "player_id",
    "display_first_last": "player_name",
    "display_last_comma_first": "player_name_last_first",
    "from_year": "from_season",
    "to_year": "to_season",
})

# Convert important columns to integers
df_players["player_id"] = df_players["player_id"].astype(int)
df_players["from_season"] = df_players["from_season"].astype(int)
df_players["to_season"] = df_players["to_season"].astype(int)

# Remove duplicate players based on player_id
df_players = df_players.drop_duplicates(subset=["player_id"])

# Show missing values for each column
missing_values = df_players.isnull().sum()
print("Missing values per column:")
print(missing_values)

df_players.head()



Missing values per column:
player_id                       0
player_name_last_first          0
player_name                     0
rosterstatus                    0
from_season                     0
to_season                       0
playercode                      0
player_slug                     0
team_id                         0
team_city                    4586
team_name                    4586
team_abbreviation            4586
team_code                    4586
team_slug                    4586
games_played_flag               0
otherleague_experience_ch       0
dtype: int64


,player_id,player_name_last_first,player_name,rosterstatus,from_season,to_season,playercode,player_slug,team_id,team_city,team_name,team_abbreviation,team_code,team_slug,games_played_flag,otherleague_experience_ch
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,NaN,NaN,NaN,NaN,NaN,Y,0
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,NaN,NaN,NaN,NaN,NaN,Y,0
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,NaN,NaN,NaN,NaN,NaN,Y,0
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,NaN,NaN,NaN,NaN,NaN,Y,0
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,NaN,NaN,NaN,NaN,NaN,Y,0


## Player Stats

In [32]:
import pandas as pd
import glob
import os

# 1. Locate all player stats CSV files in the raw directory
# The folder contains one CSV per season
# glob() finds every file matching the pattern so we can load them all.
path = "../../data/raw/player_stats/*.csv"
files = glob.glob(path)

df_player_stats_raw_list = []

# 2. Loop through each CSV file and load it
# - Extract the season from the filename 
# - Add a new column called "season" so we know which season each row belongs to
# - Append the DataFrame to a list for combining later
for file in files:
    filename = os.path.basename(file)  
    season = filename.replace("player_stats_", "").replace(".csv", "")
    
    temp_df = pd.read_csv(file)
    temp_df["season"] = season  # Add season column
    
    df_player_stats_raw_list.append(temp_df)

# 3. Combine all seasons into one DataFrame
df_player_stats_raw = pd.concat(df_player_stats_raw_list, ignore_index=True)


In [33]:
# Make a copy of the raw combined DataFrame
df_player_stats = df_player_stats_raw.copy()


# Clean column names for consistency
df_player_stats.columns = (
    df_player_stats.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("/", "_")
)

# Rename key columns to clearer names
df_player_stats = df_player_stats.rename(columns={
    "player_id": "player_id",
    "team_id": "team_id",
    "team_abbreviation": "team_abbreviation"
    # player_name already renamed above
})

# Convert important numeric columns
df_player_stats["player_id"] = df_player_stats["player_id"].astype(int)
df_player_stats["team_id"] = df_player_stats["team_id"].astype(int)
df_player_stats["age"] = df_player_stats["age"].astype(float)
df_player_stats["gp"] = df_player_stats["gp"].astype(int)
df_player_stats["w"] = df_player_stats["w"].astype(int)
df_player_stats["l"] = df_player_stats["l"].astype(int)

# Drop duplicate rows
df_player_stats = df_player_stats.drop_duplicates()

# Check missing values to confirm name column survived
missing_values = df_player_stats.isnull().sum()
print("Missing values per column:")
print(missing_values)

df_player_stats.head()


Missing values per column:
player_id                0
player_name              0
nickname                 0
team_id                  0
team_abbreviation        0
                        ..
dd2_rank                 0
td3_rank                 0
wnba_fantasy_pts_rank    0
team_count               0
season                   0
Length: 68, dtype: int64


,player_id,player_name,nickname,team_id,team_abbreviation,age,gp,w,l,w_pct,...,pf_rank,pfd_rank,pts_rank,plus_minus_rank,nba_fantasy_pts_rank,dd2_rank,td3_rank,wnba_fantasy_pts_rank,team_count,season
0,920,A.C. Green,A.C.,1610612748,MIA,37.0,82,50,32,0.610,...,220,39,212,95,218,126,26,223,1,200001
1,2062,A.J. Guyton,A.J.,1610612741,CHI,23.0,33,6,27,0.182,...,92,112,292,378,317,224,26,314,1,200001
2,243,Aaron McKie,Aaron,1610612755,PHI,28.0,76,51,25,0.671,...,316,112,87,20,84,77,4,81,1,200001
3,1425,Aaron Williams,Aaron,1610612751,NJN,29.0,82,26,56,0.317,...,441,112,91,434,77,53,26,79,1,200001
4,228,Adam Keefe,Adam,1610612744,GSW,31.0,67,14,53,0.209,...,200,112,307,385,274,224,26,280,1,200001


## Team Stats

In [34]:
import pandas as pd
import glob
import os

# 1. Locate all team stats CSV files in the raw directory
# The folder contains one CSV per season
# glob() finds every file matching the pattern so we can load them all.
path = "../../data/raw/team_stats/*.csv"
files = glob.glob(path)

df_team_stats_raw_list = []

# 2. Loop through each CSV file and load it
# - Extract the season from the filename
# - Add a new column called "season" so we know which season each row belongs to
# - Append the DataFrame to a list for combining later
for file in files:
    filename = os.path.basename(file)
    season = filename.replace("team_stats_", "").replace(".csv", "")
    
    temp_df = pd.read_csv(file)
    temp_df["season"] = season
    
    df_team_stats_raw_list.append(temp_df)

# 3. Combine all seasons into one DataFrame
df_team_stats_raw = pd.concat(df_team_stats_raw_list, ignore_index=True)



In [35]:
# Make a copy of the raw combined DataFrame
df_team_stats = df_team_stats_raw.copy()

# Clean column names for consistency
df_team_stats.columns = (
    df_team_stats.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("/", "_")
)

# Convert important numeric columns
# Only converting what actually needs to be numeric so I don't wipe out text fields
df_team_stats["team_id"] = df_team_stats["team_id"].astype(int)
df_team_stats["gp"] = df_team_stats["gp"].astype(int)
df_team_stats["w"] = df_team_stats["w"].astype(int)
df_team_stats["l"] = df_team_stats["l"].astype(int)

# Drop duplicate rows
df_team_stats = df_team_stats.drop_duplicates()

# Check missing values to confirm everything looks good
missing_values = df_team_stats.isnull().sum()
print("Missing values per column:")
print(missing_values)

df_team_stats.head()


Missing values per column:
team_id            0
team_name          0
gp                 0
w                  0
l                  0
w_pct              0
min                0
fgm                0
fga                0
fg_pct             0
fg3m               0
fg3a               0
fg3_pct            0
ftm                0
fta                0
ft_pct             0
oreb               0
dreb               0
reb                0
ast                0
tov                0
stl                0
blk                0
blka               0
pf                 0
pfd                0
pts                0
plus_minus         0
gp_rank            0
w_rank             0
l_rank             0
w_pct_rank         0
min_rank           0
fgm_rank           0
fga_rank           0
fg_pct_rank        0
fg3m_rank          0
fg3a_rank          0
fg3_pct_rank       0
ftm_rank           0
fta_rank           0
ft_pct_rank        0
oreb_rank          0
dreb_rank          0
reb_rank           0
ast_rank           0
tov_ran

,team_id,team_name,gp,w,l,w_pct,min,fgm,fga,fg_pct,...,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank,season
0,1610612737,Atlanta Hawks,82,25,57,0.305,3946.000000,2876,6668,0.431,...,28,28,16,21,27,14,8,26,25,200001
1,1610612738,Boston Celtics,82,36,46,0.439,3966.000000,2773,6485,0.428,...,19,19,3,28,17,25,22,16,20,200001
2,1610612766,Charlotte Hornets,82,46,36,0.561,3976.000000,2800,6501,0.431,...,7,8,11,11,6,11,18,24,14,200001
3,1610612741,Chicago Bulls,82,15,67,0.183,3971.000000,2721,6411,0.424,...,12,21,7,24,14,19,22,29,29,200001
4,1610612739,Cleveland Cavaliers,82,30,52,0.366,3965.613333,2890,6532,0.442,...,19,27,15,13,29,20,6,22,23,200001


## Team Game Log

In [36]:
import pandas as pd 
import glob 
import os 

# 1. Locate all team game log CSV files in the raw directory
path = "../../data/raw/team_game_logs/*.csv"
files = glob.glob(path)
df_team_game_logs_raw_list = []

# 2. Loop through each CSV file and load it
# - Extract the season from the filename
# - Add a new column called "season" so I know which season each row belongs to
for file in files:
    filename = os.path.basename(file)
    season = filename.replace("team_game_logs_", "").replace(".csv", "")
    temp_df = pd.read_csv(file)
    temp_df["season"] = season
    df_team_game_logs_raw_list.append(temp_df)

# 3. Combine all seasons into one DataFrame
df_team_game_logs_raw = pd.concat(df_team_game_logs_raw_list, ignore_index=True)

In [37]:
# Make a copy of the raw combined DataFrame
df_team_game_logs = df_team_game_logs_raw.copy()

# Clean column names for consistency
df_team_game_logs.columns = (
    df_team_game_logs.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("/", "_")
)

# Convert important numeric columns
# Only converting what actually needs to be numeric so I don't wipe out text fields
df_team_game_logs["team_id"] = df_team_game_logs["team_id"].astype(int)
df_team_game_logs["game_id"] = df_team_game_logs["game_id"].astype(int)

# Convert game date to datetime
df_team_game_logs["game_date"] = pd.to_datetime(df_team_game_logs["game_date"])

# Drop duplicate rows
df_team_game_logs = df_team_game_logs.drop_duplicates()

# Check missing values to confirm everything looks good
missing_values = df_team_game_logs.isnull().sum()
print("Missing values per column:")
print(missing_values)

df_team_game_logs.head()


Missing values per column:
season_id            0
team_id              0
team_abbreviation    0
team_name            0
game_id              0
game_date            0
matchup              0
wl                   2
min                  0
fgm                  0
fga                  0
fg_pct               2
fg3m                 0
fg3a                 0
fg3_pct              2
ftm                  0
fta                  0
ft_pct               3
oreb                 0
dreb                 0
reb                  0
ast                  0
stl                  0
blk                  0
tov                  0
pf                   0
pts                  0
plus_minus           0
video_available      0
season               0
dtype: int64


,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,fgm,...,reb,ast,stl,blk,tov,pf,pts,plus_minus,video_available,season
0,22000,1610612737,ATL,Atlanta Hawks,20000004,2000-10-31,ATL vs. CHH,L,240,30,...,29,14,9,2,13,31,82,-24,0,200001
1,22000,1610612741,CHI,Chicago Bulls,20000006,2000-10-31,CHI vs. SAC,L,240,26,...,29,19,8,4,20,20,81,-19,0,200001
2,22000,1610612744,GSW,Golden State Warriors,20000011,2000-10-31,GSW vs. PHX,W,240,32,...,55,18,11,5,21,22,96,2,0,200001
3,22000,1610612746,LAC,Los Angeles Clippers,20000010,2000-10-31,LAC @ UTA,L,240,34,...,44,18,3,6,24,32,94,-13,0,200001
4,22000,1610612749,MIL,Milwaukee Bucks,20000007,2000-10-31,MIL @ DAL,L,240,33,...,51,16,6,7,20,27,93,-4,0,200001


### Check Missing ID Columns

In [39]:
df_players["player_id"].isnull().sum()
df_player_stats["player_id"].isnull().sum()
df_team_stats["team_id"].isnull().sum()
df_team_game_logs["team_id"].isnull().sum()
df_team_game_logs["game_id"].isnull().sum()


np.int64(0)

## Save Processed Data

In [40]:
# Save cleaned players table
df_players.to_csv("../../data/processed/players.csv", index=False)

# Save cleaned player stats table
df_player_stats.to_csv("../../data/processed/player_stats.csv", index=False)

# Save cleaned team stats table
df_team_stats.to_csv("../../data/processed/team_stats.csv", index=False)

# Save cleaned team game logs table
df_team_game_logs.to_csv("../../data/processed/team_game_logs.csv", index=False)

print("All cleaned DataFrames saved to data/processed/")


All cleaned DataFrames saved to data/processed/
